In [62]:
import pandas as pd

# convert the excel to a CSV because it's easier to look at
df_excel = pd.read_excel("data/raw_data/LichenMossdata.xlsx")
df_excel.to_csv("data/raw_data/lichen_moss_coverage_data.csv")

df_raw = pd.read_csv("data/raw_data/lichen_moss_coverage_data.csv")
df_processed = df_raw

In [63]:
# the "total coverage" is the total amount of lichen AND moss covering the
# surface of a rock
# the "moss fraction" is the fraction OF THE TOTAL that is moss.
# so it's a percentage of a percentage (we recoreded it that way because
# it was methodologically easier)
# example: 10% total cover, 40% moss cover
# 40% of 10% = 4%
# therefore there is 4% moss, and the 6% remaining is lichen
# the code below processes our values into simply "moss coverage" and
# "lichen coverage"

# the cover values are %, so divide by 100 to get the true value
total_cover = df_raw['total_cover'] / 100
moss_fraction = df_raw['moss_fraction'] / 100
moss_cover = total_cover * moss_fraction
lichen_cover = total_cover - moss_cover

df_processed['total_cover'] = total_cover
df_processed['moss_cover'] = moss_cover
df_processed['lichen_cover'] = lichen_cover


In [64]:
# convert all the "< 0.5" (= below detection threshold) values to the middle value
# "< 0.5" -> "0.25"

def handle_below_detection(item):
    # for any item that starts with "<"
    if isinstance(item, str) and "<" in item:
         # take the number part of the string and convert to a float
         num = float(item.split("<")[1])
         return num / 2
    return item

# doing this the barbaric way :)
df_processed['no2'] = df_raw['no2'].apply(handle_below_detection)
df_processed['no3'] = df_raw['no3'].apply(handle_below_detection)
df_processed['nh4'] = df_raw['nh4'].apply(handle_below_detection)
df_processed['total_nitrogen'] = df_raw['total_nitrogen'].apply(handle_below_detection)
df_processed['po4'] = df_raw['po4'].apply(handle_below_detection)    

In [65]:
# convert "has lichen type" columns to bools
df_processed['has_crustose'] = df_raw['crustose'].astype(bool)
df_processed['has_foliose'] = df_raw['foliose'].astype(bool)
df_processed['has_fruticose'] = df_raw['fruticose'].astype(bool)

df_processed = df_processed.drop(['crustose', 'foliose', 'fruticose'], axis=1)

In [66]:
df_raw['species']

0                  G_c,O_c,W_c,Bl_f
1                  Bl_f,W_c,G_c,O_c
2                           W_c,G_c
3                           W_c,O_c
4                       G_c,W_c,O_c
5                   O_c,W_c,G_c,W_f
6              O_c,W_c,G_c,W_f,Bl_f
7                  G_c,O_c,W_c,GR_c
8                          G_c,W_c 
9             W_c,W_fr,W_f,G_c,GR_c
10                W_fr,G_c,W_c,GR_c
11                      W_f,G_c,W_c
12       W_c,W_c,W_fr,Bl_f,G_c,GR_c
13        W_f,W_c,W_fr,Bl_c,G_c,W_c
14                 W_c,G_c,Bl_c,O_c
15             W_c,W_c,W_fr,W_f,G_c
16            W_fr,W_c,W_f,G_c,GR_c
17            W_fr,W_f,W_c,GR_c,G_c
18                W_fr,W_c,G_c,GR_c
19                W_fr,W_c,G_c,Bl_f
20                  Y_f,G_c,W_c,W_f
21            G_c,W_fr,W_c,W_f,GR_c
22                  G_c,R_c,W_c,O_c
23                      G_c,W_c,O_c
24                          W_c,G_c
25             G_c,W_c,O_c,Bl_c,W_f
26                     G_c,W_c,Bl_c
27                      G_c,

In [67]:
# convert the strings in "species" to lists of strings
species = pd.Series([species.strip().split(',') for species in df_raw['species']])
df_processed['species'] = species


In [68]:
# get a set with all the unique species we have observed
all_species = set()
for species_list in df_processed['species']:
    for unique_species in species_list:
        all_species.add(unique_species)
print(f'all species: {all_species}')

# use the set to make new columns for each observed species, "has_{species}"
for species in all_species:
    df_processed[f'has_{species}'] = False

# on each column determine if a unique species is present or not
for index, row in df_processed.iterrows():
    for unique_species in row['species']:
        df_processed.loc[index, f'has_{unique_species}'] = True

all species: {'W_c', 'W_f', 'R_c', 'W_fr', 'Bl_c', 'Y_f', 'Bl_f', 'G_c', 'GR_c', 'O_c'}


In [69]:
df_processed['has_foliose_or_fruticose'] = False
for index, row in df_processed.iterrows():
    if row['has_foliose'] == True or row['has_fruticose'] == True:
        df_processed.loc[index, 'has_foliose_or_fruticose'] = True

In [70]:
# drop redundant columns
df_processed = df_processed.drop(columns=['moss_fraction'])
# drop nan values
df_processed = df_processed.dropna()

In [71]:
df_processed.to_csv('data/processed_data/coverage_data_processed.csv')